In [24]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import sklearn
from sklearn.impute import KNNImputer,SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from tqdm import tqdm
from sklearn.metrics import f1_score
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import make_scorer

from sklearn.preprocessing import StandardScaler, RobustScaler
import warnings

warnings.filterwarnings('ignore')

In [25]:


info = {
    "MUSTERI_ID" : "Müşterileri ayırt eden unique ID'ler.",
    "LABEL": "Müşterinin aldığı Hayat sigortası ürününün çeşidi.",
    "FLAG":"Verinin ait olduğu ay.",
    "PP_CINSIYET": "Sözleşme sahibinin cinsiyeti.1: Erkek 2: Kadın",
    "PP_YAS" : "Sözleşme sahibinin ay bazlı olarak yaşı.",
    "PP_MESLEK" : "Sözleşme sahibinin mesleği.",
    "PP_MUSTERI_SEGMENTI" : "Sözleşme sahibinin müşteri segmenti. 101: A segment, 102: B segment,103: C segment, 104: D segment,105: E segment ,106: F segment",
    "PP_UYRUK": "Sözleşme sahibinin uyruk bilgisi. 1:TC Vatandaşı , 2:Mavi Kart, 3:Yabancı Uyruklu",
    "IL":"Sözleşme sahiplerinin yaşadığı illere ait plaka kodu.( 0 = Yurtdışı)",
    "SORU_YATIRIM_KARAKTERI_CVP": "Sözleşme sahibinin ankete verdiği cevaplara göre belirlenen yatırım karakteri.",
    "SORU_YATIRIM_KARAKTERI_RG": "Sözleşme sahibinin ankette verdiği cevabın üstünden geçen süre (ay).",
    "SORU_MEDENI_HAL_CVP": "Sözleşme sahibinin ankete verdiği cevaba göre medeni durumu.",
    "SORU_MEDENI_HAL_RG":" Sözleşme sahibinin ankette verdiği cevabın üstünden geçen süre (ay).",
    "SORU_EGITIM_CVP": "Sözleşme sahibinin ankete verdiği cevaplara göre eğitim durumu.",
    "SORU_EGITIM_RG": "Sözleşme sahibinin ankette verdiği cevabın üstünden geçen süre (ay).",
    "SORU_GELIR_CVP": "Sözleşme sahibinin ankete verdiği cevaba göre gelir durumu.",
    "SORU_GELIR_RG": "Sözleşme sahibinin ankette verdiği cevabın üstünden geçen süre (ay).",
    "SORU_COCUK_SAYISI_CVP": "Sözleşme sahibinin ankete verdiği cevaba göre çocuk sayısı.",
    "SORU_COCUK_SAYISI_RG": "Sözleşme sahibinin ankette verdiği cevabın üstünden geçen süre (ay).",
    "BES_AYRILMA_TALEP_ADET" : "Sözleşme sahibinin BES hesabından ayrılmak için açtığı talep sayısı.",
    "ODEMEME_TALEP_ADET" : "Sözleşme sahibinin son 1 sene içerisinde kaç kez ödememe talimatı verdiğini gösterir.",
    "HAYAT_AYRILMA_TALEP_ADET" : "Sözleşme sahibinin Hayat sigortasından ayrılmak için açtığı talep sayısı.",
    "BILGI_TALEP_ADET" : "Sözleşme sahibinin sözleşmesi için kaç kez bilgi talep ettiğini gösterir.",
    "VADE_TUTAR_0 - VADE_TUTAR_11" : "Sözleşme sahibinin sahip olduğu ürünlerin son 12 aya ait toplam vade tutarları",
    "ODEME_TUTAR_0 - ODEME_TUTAR_11 ": "Sözleşme sahibinin sahip olduğu ürünler için son 12 ayda yaptığı ödeme tutarları",
    "SON_AY_KATKI_MIKTARI" : "Sözleşme sahibinin son bir ay içinde yaptığı ek katkı ödemelerinin TL cinsinden toplam miktarı",
    "SON_AY_KATKI_ADET" : "Sözleşme sahibinin son bir ay içinde yaptığı ek katkı ödemelerinin adedi",
    "SON_CEYREK_KATKI_MIKTARI" : "Sözleşme sahibinin son üç ay içinde yaptığı ek katkı ödemelerinin TL cinsinden toplam miktarı",
    "SON_CEYREK_KATKI_ADET" : "Sözleşme sahibinin son üç ay içinde yaptığı ek katkı ödemelerinin adedi",
    "SON_SENE_KATKI_MIKTARI" : "Sözleşme sahibinin son bir sene içinde yaptığı ek katkı ödemelerinin TL cinsinden toplam miktarı",
    "SON_SENE_KATKI_ADET" : "Sözleşme sahibinin son bir sene içinde yaptığı ek katkı ödemelerinin adedi",
    "ANAPARA": "Sözleşme sahibinin TL cinsinden toplam yatırdığı para miktarı.",
    "GETIRI" : "Sözleşme sahibinin TL cinsinden yatirdiği paradan elde ettiği getiri.",
    "BU1 - BU24": "BES ÜRÜN 1 - 24, Kişinin kolonda belirtilen BES ürününe sahip olup olmama durumu (Ürün özellikleri ayrı bir dokumanda verilmiştir)",
    "HU1 - HU19": "HAYAT ÜRÜN 1 - 19, Kişinin kolonda belirtilen Hayat ürününe sahip olup olmama durumu (Ürün özellikleri ayrı bir dokumanda verilmiştir)",
    "AKTIF_ILK_POLICE_RG": "Sözleşme sahibinin aktif olan poliçeleri arasından en eskisinin üstünden geçen süre (ay)."

}


class Analysies():
    def __init__(self,trainPath,testPath,excelPath, info= info):

        self.trainData = pd.read_csv(trainPath)
        self.testData = pd.read_csv(testPath)
        self.productFeature = pd.read_excel(excelPath)
        self.info = info

    def grab_col_names(self,df,cat_th=20, car_th=25):
        """

        Veri setindeki kategorik, numerik ve kategorik fakat kardinal değişkenlerin isimlerini verir.
        Not: Kategorik değişkenlerin içerisine numerik görünümlü kategorik değişkenler de dahildir.

        Parameters
        ------
            df: df
                    Değişken isimleri alınmak istenilen df
            cat_th: int, optional
                    numerik fakat kategorik olan değişkenler için sınıf eşik değeri
            car_th: int, optinal
                    kategorik fakat kardinal değişkenler için sınıf eşik değeri

        Returns
        ------
            cat_cols: list
                    Kategorik değişken listesi
            num_cols: list
                    Numerik değişken listesi
            cat_but_car: list
                    Kategorik görünümlü kardinal değişken listesi

        Examples
        ------
            import seaborn as sns
            df = sns.load_dataset("iris")
            print(grab_col_names(df))


        Notes
        ------
            cat_cols + num_cols + cat_but_car = toplam değişken sayısı
            num_but_cat cat_cols'un içerisinde.
            Return olan 3 liste toplamı toplam değişken sayısına eşittir: cat_cols + num_cols + cat_but_car = değişken sayısı

        """


    # cat_cols, cat_but_car
        cat_cols = [col for col in df.columns if df[col].dtypes == "O"]
        num_but_cat = [col for col in df.columns if df[col].nunique() < cat_th and
                    df[col].dtypes != "O"]
        cat_but_car = [col for col in df.columns if df[col].nunique() > car_th and
                    df[col].dtypes == "O"]
        cat_cols = cat_cols + num_but_cat
        cat_cols = [col for col in cat_cols if col not in cat_but_car]

        # num_cols
        num_cols = [col for col in df.columns if df[col].dtypes != "O"]
        num_cols = [col for col in num_cols if col not in num_but_cat]
        
       
       


        print(f"Observations: {df.shape[0]}")
        print(f"Variables: {df.shape[1]}")
        print(f'cat_cols: {len(cat_cols)}')
        print(f'num_cols: {len(num_cols)}')
        print(f'cat_but_car: {len(cat_but_car)}')
        print(f'num_but_cat: {len(num_but_cat)}')
        return cat_cols, num_cols, cat_but_car
    def cat_summary(self,dataframe, col_name, plot=False):

        for col in col_name:

            
            if col.startswith("BU") is False or col.startswith("HU") is False :
                
                print(pd.DataFrame({col: dataframe[col].value_counts(),
                                    "Ratio": 100 * dataframe[col].value_counts() / len(dataframe),
                                    "Info" : self.info[col]}))
                
                print(dataframe[col].dtype)

                print("##########################################")
                
            else:
                print(pd.DataFrame({col: dataframe[col].value_counts(),
                                    "Ratio": 100 * dataframe[col].value_counts() / len(dataframe),
                                    "Info" : self.info["BU1 - BU24"]}))

            if plot:
                
                if col.startswith("BU") is False or col.startswith("HU") is False :
                    groupped = dataframe.groupby(col,as_index=True)[col].count()

                    

                    fig = go.Figure(go.Bar(
                        x=groupped.index,
                        y = groupped.values),
                        layout_title_text= info[col]
                    )
                    fig.show()
                    # plt.figure(figsize=(7,7))
                    # sns.countplot(x=dataframe[col], data=dataframe)
                    

                    # plt.title(info[col])
                    # plt.show(block=True)
                else:
                    groupped = dataframe.groupby(col,as_index=True)[col].count()

                    

                    fig = go.Figure(go.Bar(
                        x=groupped.index,
                        y = groupped.values),
                        layout_title_text= info["BU1 - BU24"]
                    )
                    fig.show()
                    
                    # plt.figure(figsize=(10,10))
                    # sns.countplot(x=dataframe[col], data=dataframe)
                    

                    # plt.title(info["BU1 - BU24"])
                    # plt.show(block=True)
                

In [ ]:
anl = Analysies("/kaggle/input/train-dataset/train.csv",
                "/kaggle/input/test-dataset/test.csv",
                "/kaggle/input/urun-ozellikleri/Urun Ozellikleri.xlsx",
                info)


anl.trainData['SORU_GELIR_CVP'] = anl.trainData['SORU_GELIR_CVP'].apply(lambda x : x == x.split(",")[0] + "." + x.split(",")[1] if type(x) is str and ( ",") in list(x) else x)  
anl.trainData['SORU_GELIR_CVP'] = anl.trainData['SORU_GELIR_CVP'].astype(np.float32)



anl.testData['SORU_GELIR_CVP'] = anl.testData['SORU_GELIR_CVP'].apply(lambda x : x == x.split(",")[0] + "." + x.split(",")[1] if type(x) is str and (",") in list(x) else x)  
anl.testData['SORU_GELIR_CVP'] = anl.testData['SORU_GELIR_CVP'].astype(np.float32)







In [ ]:
cat_cols, num_cols, cat_but_car = anl.grab_col_names(anl.trainData)


In [ ]:
cat_cols.remove("LABEL")

In [ ]:
x_data = anl.trainData.drop(["LABEL"],axis = 1)
changed_data = x_data.drop(cat_but_car,axis=1)
y_data = anl.trainData[["LABEL"]]
def dropSomeCols(df:pd.DataFrame):

    cols = []

    for col in df.columns:
        info = df[col].isna().value_counts()
        info2 = pd.DataFrame({str(col) : [info.values[i]] for i,col in enumerate(info.index.values)})
        if  info2["False"].values < (int(len(df[col])/4)):
            cols.append(col)

    new_df = df.drop(cols,axis = 1)

    print(f"{cols} are dropped from df because almost all data is nan")

    return new_df,cols

droppedDf,cols = dropSomeCols(changed_data)

new_cat,new_num,new_cat_but_car = anl.grab_col_names(droppedDf)


num_keys = ["ADET","GELIR","MIKTARI","SAYISI"]





In [ ]:
for col in new_cat:
    try:


        splitted = col.split("_")
        if (splitted[1] in num_keys):
            new_num.append(col)
            new_cat.remove(col)
        elif (splitted[-1] in num_keys):
            new_num.append(col)
            new_cat.remove(col)
        elif (splitted[-2] in num_keys):
            new_num.append(col)
            new_cat.remove(col)
    except:
        splitted = col
        continue

print(new_cat)

In [ ]:
test_data = anl.testData
cat_data = test_data[new_cat]
num_data = test_data[new_num]

for col in new_cat:

    cat_data[col] = cat_data[col].astype("O")

for col in new_cat:
    cat_data[col] = cat_data[col].fillna(cat_data[col].mode().iloc[0])



In [ ]:
knnImpute = SimpleImputer(strategy = "mean")
imputed = knnImpute.fit_transform(droppedDf[new_num])



In [ ]:
new_df = pd.DataFrame({cols : imputed.T[i] for i,cols in enumerate(new_num)})
x_data = droppedDf.drop(new_df.columns,axis = 1)
x_data = pd.concat([new_df,x_data],axis = 1)


In [ ]:


for col in new_cat:
    x_data[col] = x_data[col].fillna(x_data[col].mode().iloc[0])


    
    

In [ ]:
def outlier_thresholds(df, col_name, q1=0.33, q3=0.66):
    quartile1 = df[col_name].quantile(q1)
    quartile3 = df[col_name].quantile(q3)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

def replace_with_thresholds(x_data,y_data,variable):
    low_limit, up_limit = outlier_thresholds(x_data, variable)
    
    indexes_list = []
    
    
    if len(x_data.loc[(x_data[variable] < low_limit), variable].index.values) != 0 :
        
        indexes_list.extend(list(x_data.loc[(x_data[variable] < low_limit), variable].index.values))
    elif len(x_data.loc[(x_data[variable]>up_limit),variable].index.values) !=0:
        indexes_list.extend(list(x_data.loc[(x_data[variable]>up_limit),variable].index.values))
        
    
    if len(indexes_list)>0:
        
    
        
        willDelete = y_data.iloc[indexes_list] == "UA"
        
        willDelete = list(willDelete[willDelete["LABEL"] == True].index.values)
        
        print("#################################")
        
        
        x_data = x_data.drop(willDelete,axis=0).reset_index(drop=True)
        y_data = y_data.drop(willDelete,axis = 0).reset_index(drop = True)
        
        
    x_data.loc[(x_data[variable] < low_limit), variable] = low_limit
    x_data.loc[(x_data[variable] > up_limit), variable] = up_limit
    
    indexes_list.clear()
    
    return x_data,y_data
    
    
    


def check_outlier(df, col_name):
    low_limit, up_limit = outlier_thresholds(df, col_name)
    if df[(df[col_name] > up_limit) | (df[col_name] < low_limit)].any(axis=None):
        
        print(f"{col_name} : True")
        return True
    else:
        print(f"{col_name} : False")
        return False
    


In [ ]:
for col in new_num:
    print(col)
    x_data,y_data = replace_with_thresholds(x_data,y_data,col)

In [ ]:
imputed_test = knnImpute.transform(num_data)
imputed_df_test = pd.DataFrame({col : imputed_test.T[i] for i,col in enumerate(new_num)})




In [ ]:

for col in new_cat:

    x_data[col] = x_data[col].astype("O")



ohe = OneHotEncoder(handle_unknown="ignore",sparse=False)


transformed_x = ohe.fit_transform(x_data[new_cat])
print(transformed_x)


encoded_x = pd.DataFrame({col : transformed_x.T[i] for i,col in enumerate(ohe.get_feature_names_out(new_cat))})





In [ ]:
for col in new_cat:
    cat_data[col] = cat_data[col].fillna(cat_data[col].mode().iloc[0])

    

transformed_test = ohe.fit_transform(cat_data)
encoded_test = pd.DataFrame({col : transformed_test.T[i] for i,col in enumerate(ohe.get_feature_names_out(cat_data.columns))})


In [ ]:

le = LabelEncoder()

y_data = le.fit_transform(y_data)

new_x = x_data.drop(new_cat,axis= 1)
new_x = pd.concat([new_x,encoded_x],axis = 1)
new_x = new_x.drop(new_cat_but_car,axis = 1)


scaler = RobustScaler()


scaled_x = scaler.fit_transform(new_x[new_num],y_data)
new_x.drop(new_num,axis=1,inplace=True)
scaled_x = pd.DataFrame({cols : scaled_x.T[i] for i,cols in enumerate(new_num)})

new_x = pd.concat([new_x,scaled_x],axis=1)






In [ ]:
num_test_scaled = scaler.transform(imputed_df_test)

num_test_scaled_df = pd.DataFrame({cols : num_test_scaled.T[i] for i,cols in enumerate(new_num)})

processed_test_data = pd.concat([encoded_test,num_test_scaled_df],axis = 1)


In [ ]:
cat_test_new, num_test_new, car_test_new = anl.grab_col_names(processed_test_data)
cat_train_new,num_train_new,car_train_new = anl.grab_col_names(new_x)

will_add_in_train = [col  for col in cat_test_new if col not in cat_train_new]
will_add_in_test = [col for col in cat_train_new if col not in cat_test_new ]




In [ ]:
print("####################################")

print(f"Columns which will add in traindata {will_add_in_train}")
print("####################################")
print(f"Columns which will add in testdata {will_add_in_test}")

zero_matrix_train = np.zeros((len(new_x),1))
zero_matrix_test = np.zeros((len(processed_test_data),1))


for col in will_add_in_train:
    new_x[col] = zero_matrix_train

for col in will_add_in_test:
    processed_test_data[col] = zero_matrix_test

processed_test_data = processed_test_data[new_x.columns]

In [ ]:

# num_classes = anl.trainData["LABEL"].nunique()

# param_grid = {
#     "XGBM": {
#         "objective" : ["multi:softprob"],
#         'gamma': [0.5, 1, 1.5, 2, 5],
#         "num_class" : [num_classes],
#         "booster" : ["gbtree","dart"],
#         "eta" : np.linspace(1e-3,0.1,3),
#         "max_depth" :  np.arange(5,18,4),
#         "subsample" : np.linspace(0.5,1.,3),
#         "colsample_bytree" : np.linspace(0.5,1.,3),
#         "n_estimators" : [180,350,500,800]
#     },

#     "LGBM" : {
#         "objective" : ["multi_class"],
#         "boosting_type" : ["gbdt","goss","dart"],
#         "max_depth" : np.arange(5,18,4),
#         "colsample_bytree" :  np.linspace(0.5,1.,3),
#         "subsample" : np.linspace(0.5,1.,3),
#         "learning_rate" : np.linspace(1e-3,0.1,4),
#         # "class_weight" : {[0,1,2,3,4,5,6,7] : []},
#         "reg_lambda" : np.linspace(0.,1.,3),
#         "importance_type" : ["split","gain"],
#         "num_classes" : [num_classes]
        
        
#         }
 
# }


# models = {
#     "LGBM" : LGBMClassifier(),
#     "XGBM" : XGBClassifier()
    
# }

classes = [0, 1, 2, 3, 4, 5, 6, 7]
weights = [0.0385, 0.0328, 0.2791, 0.1812, 0.0113, 0.2952, 0.1614, 0.0001]

def competition_metric(y_true, y_pred):
    
    return np.sum(f1_score(y_true, y_pred, average=None, labels=classes) * weights)


# bests = {

#     }

# def train(params:dict,model,trainX:pd.DataFrame,trainY:pd.DataFrame):

    

        
#     cv = RandomizedSearchCV(models[model],params,scoring= custom_scorer,cv = 5,refit = True, verbose = 10,error_score = "raise").fit(trainX,trainY)
    

    
            
            
        
#     return cv.best_params_



# for model in list(models.keys()):

#     bests[model] = train(params = param_grid[model],model = model,trainX = new_x, trainY=y_data)







In [ ]:
bests = {'subsample': 0.5,
  'reg_lambda': 1.0,
  'objective': 'multiclass',
  'num_class': 8,
  'max_depth': 17,
  'learning_rate': 0.034,
  'importance_type': 'split',
  'colsample_bytree': 0.75,
  'boosting_type': 'goss'}

# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier()

# Train the Random Forest classifier on the training data
rf_classifier.fit(new_x, y_data)

# Predict the labels for the test set
y_pred = rf_classifier.predict(processed_test_data)

In [ ]:
submission = pd.read_csv(r"/kaggle/input/sample-submission/sample_submission.csv")

final_submission = pd.concat([anl.testData["MUSTERI_ID"], pd.DataFrame(y_pred, columns=['LABEL'])], axis=1)

# Define the mapping dictionary
label_mapping = {
    0: 'HU06',
    1: 'HU07',
    2: 'HU11',
    3: 'HU12',
    4: 'HU14',
    5: 'HU15',
    6: 'HU19',
    7: 'UA'
}

# Convert the "LABEL" column to string labels
final_submission['LABEL'] = final_submission['LABEL'].map(label_mapping).astype(str)

final_submission.to_csv(r'/kaggle/working/submission.csv', index=False)